In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd

ticker = 'AAPL'
start_date = '2023-01-01'
end_date = '2024-12-31'
data = yf.download(ticker, start=start_date, end=end_date)

data.to_csv('apple_daily.csv')

weekly_data = data.resample('W').last()
weekly_data.to_csv('apple_weekly.csv')

biweekly_data = data.resample('2W').last()
biweekly_data.to_csv('apple_biweekly.csv')


[*********************100%***********************]  1 of 1 completed


In [4]:
import pandas as pd 

df = pd.read_csv("apple_daily.csv")
df.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,123.63252258300781,129.39551044224086,122.74286511723291,128.7826411649236,112117500
3,2023-01-04,124.9076919555664,127.18126010449132,123.64240474081771,125.43159923588829,89113600
4,2023-01-05,123.58309936523438,126.30149268134268,123.32609306355886,125.66884902202133,80962700
